# Models to predict ratings

In [24]:
import pandas as pd
import numpy as np

friends = pd.read_csv('transcripts and metadata/friends_mastersheet.csv', index_col=0)

In [25]:
friends.head()

,Year_of_prod,Season,Episode Number,Episode_Title,Duration,Summary,Director,Stars,Votes,match,...,Phoebe,Ross,Rachel,Carol,Susan,Janice,Mike,Gunther,Ben,Emily
0,1994,1,1,The One Where Monica Gets a Roommate: The Pilot,22,"Monica and the gang introduce Rachel to the ""r...",James Burrows,8.3,7440,11,...,0.061248,0.144114,0.155643,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,1994,1,2,The One with the Sonogram at the End,22,Ross finds out his ex-wife is pregnant. Rachel...,James Burrows,8.1,4888,12,...,0.061500,0.248342,0.165269,0.088068,0.044696,0.000000,0.0,0.0,0.0,0.0
2,1994,1,3,The One with the Thumb,22,Monica becomes irritated when everyone likes h...,James Burrows,8.2,4605,13,...,0.145415,0.125624,0.102383,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,1994,1,4,The One with George Stephanopoulos,22,Joey and Chandler take Ross to a hockey game t...,James Burrows,8.1,4468,14,...,0.111161,0.151849,0.162042,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,1994,1,5,The One with the East German Laundry Detergent,22,"Eager to spend time with Rachel, Ross pretends...",Pamela Fryman,8.5,4438,15,...,0.095007,0.150061,0.141764,0.000000,0.000000,0.047483,0.0,0.0,0.0,0.0


In [26]:
resultlist = [['model', 'LOOCV mean score', 'std']]
books = pd.read_csv('books/friends_books_topics_model.csv', index_col=0)
sb = pd.read_csv('scriptbase/friends_scriptbase_topics.csv', index_col=0)
friends_no_topics = pd.read_csv('transcripts and metadata/friends_mastersheet.csv', index_col=0)

## Baselines

### dummy baseline

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

cv = LeaveOneOut()

X = friends.text
y = friends.Stars

vect = TfidfVectorizer(max_features=5000)
reg = DummyRegressor(strategy="mean")
pipe = make_pipeline(vect, reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

resultlist.append(['dummy_baseline', np.mean(scores), np.std(scores)])

print(np.mean(scores), np.std(scores))

-0.3113667508113955 0.24655333378118335


### BOW baseline

In [28]:
from sklearn.svm import SVR

cv = LeaveOneOut()

X = friends.text
y = friends.Stars

vect = TfidfVectorizer(max_features=5000)
reg = SVR()
pipe = make_pipeline(vect, reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

resultlist.append(['BOW_baseline', np.mean(scores), np.std(scores)])

print(np.mean(scores), np.std(scores))

-0.31038842338807143 0.2511842652329307


In [29]:
model = pipe.fit(X, y)

### No topics baseline

In [30]:
lst = ['Gary Halvorson',        
'Kevin Bright',          
'Michael Lembeck',       
'James Burrows',           
'Gail Mancuso',           
'Peter Bonerz',            
'David Schwimmer',         
'Ben Weiss']

def direr(wow):
    if wow not in lst:
        wow = wow.replace(wow,'Other')
    return wow

In [31]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

X = friends_no_topics.drop(columns=['Summary', 'Episode_Title', 'Duration', 'Stars', 'Votes', 'text', 'match', 'len in words', 'len_prep'])
X.Director = X.Director.apply(direr)
y = friends_no_topics.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))
resultlist.append(['no_topic_baseline', np.mean(scores), np.std(scores)])

model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.3200979000339483 0.2584603535264025


### features + tf-idf, no topics

In [32]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import TfidfVectorizer

X = friends_no_topics.drop(columns=['Summary', 'Episode_Title', 'Stars', 'Duration', 'Votes', 'match', 'len in words', 'len_prep'])
X.Director = X.Director.apply(direr)
y = friends_no_topics.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()
vect = TfidfVectorizer(max_features=5000)

categorical_features = [X.select_dtypes(include="object").columns[0]]
integer_features= X.select_dtypes(exclude="object").columns
text_features = X.select_dtypes(include="object").columns[1]

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features),
                                (vect, text_features),
                               remainder='passthrough')

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))
resultlist.append(['no_topic_tf_idf_baseline', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.31056740251808684 0.2547505392051503


## Models all features

### Books corpus

In [33]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

X = books.drop(columns=['Summary', 'Episode_Title', 'Stars', 'Duration', 'Votes', 'text', 'match', 'len in words', 'len_prep'])
X.Director = X.Director.apply(direr)
y = books.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['books_all_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# # feature_names = model[:-1].get_feature_names_out()

# # X_preprocessed = pd.DataFrame(
# #     model[:-1].transform(X), columns=feature_names
# # )

# # coefs = pd.DataFrame(
# #     model[-1].coef_* X_preprocessed.std(axis=0),
# #     columns=["Coefficients"],
# #     index=feature_names,
# # )

# # coefs["Coefficients"].sort_values(ascending=False)

-0.31670502401903533 0.25496376558355


### scriptbase

In [34]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

X = sb.drop(columns=['Summary', 'Episode_Title', 'Votes', 'Duration', 'Stars', 'text', 'match', 'len in words', 'len_prep'])
X.Director = X.Director.apply(direr)
y = sb.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['sc_all_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.3099117990860839 0.24303544586694656


## models 50

### books

In [35]:
books_best_50 = ['t83', 't188', 't274', 't230', 't273', 't204', 't196',
       't176', 't7', 't120', 't151', 't258', 't139', 't70', 't110', 't97',
       't235', 't282', 't104', 't62', 't245', 't80', 't171', 't115', 't270',
       't109', 't150', 't269', 't158', 't203', 't59', 't295', 't175', 't51',
       't166', 't113', 't285', 't299', 't286', 't88', 't127', 't69', 't222',
       't272', 't34', 't58', 't153', 't271', 't259', 't73']

to_drop = [f't{i}' for i in range(0,300) if f't{i}' not in books_best_50]

In [36]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

columns = ['Summary', 'Episode_Title', 'Stars', 'Duration', 'Votes', 'text', 'match', 'len in words', 'len_prep'] + to_drop

X = books.drop(columns=columns)
X.Director = X.Director.apply(direr)
y = books.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['books_50_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.3025465625466021 0.2652608993937459


### scriptbase

In [37]:
sb_best_50 = ['t134', 't103', 't12', 't194', 't276', 't35', 't180',
       't222', 't32', 't191', 't299', 't64', 't202', 't288', 't275', 't260',
       't84', 't182', 't165', 't210', 't90', 't204', 't255', 't217', 't239',
       't205', 't26', 't86', 't7', 't119', 't296', 't67', 't265', 't152',
       't128', 't72', 't250', 't242', 't287', 't190', 't57', 't44', 't17',
       't132', 't189', 't240', 't193', 't21', 't138', 't130']

to_drop = [f't{i}' for i in range(0,300) if f't{i}' not in sb_best_50]

In [38]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

columns = ['Summary', 'Episode_Title', 'Stars', 'Duration', 'Votes', 'text', 'match', 'len in words', 'len_prep'] + to_drop

X = sb.drop(columns=columns)
X.Director = X.Director.apply(direr)
y = books.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['sb_50_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.28075964835406053 0.22891460909066416


## models 25

### books

In [39]:
books_best_25 = ['t83', 't188', 't274', 't230', 't273', 't204', 't196',
't176', 't7', 't120', 't151', 't258', 't139', 't70', 't110', 't97',
't235', 't282', 't104', 't62', 't245', 't80', 't171', 't115', 't270']

to_drop = [f't{i}' for i in range(0,300) if f't{i}' not in books_best_25]

In [40]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

columns = ['Summary', 'Episode_Title', 'Stars', 'Duration', 'Votes', 'text', 'match', 'len in words', 'len_prep'] + to_drop

X = books.drop(columns=columns)
X.Director = X.Director.apply(direr)
y = books.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['books_25_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.3098606761309716 0.24948065582815313


### scriptbase

In [41]:
sb_best_25 = ['t134', 't103', 't12', 't194', 't276', 't35', 't180',
       't222', 't32', 't191', 't299', 't64', 't202', 't288', 't275', 't260',
       't84', 't182', 't165', 't210', 't90', 't204', 't255', 't217', 't239']

to_drop = [f't{i}' for i in range(0,300) if f't{i}' not in sb_best_25]

In [42]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

columns = ['Summary', 'Episode_Title', 'Stars', 'Duration', 'Votes', 'text', 'match', 'len in words', 'len_prep'] + to_drop

X = sb.drop(columns=columns)
X.Director = X.Director.apply(direr)
y = books.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['sb_25_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.2834794060697638 0.2279225403433374


## Models 10

### books

In [43]:
sb_best_25 = ['t134', 't103', 't12', 't194', 't276', 't35', 't180',
       't222', 't32', 't191', 't299', 't64', 't202', 't288', 't275', 't260',
       't84', 't182', 't165', 't210', 't90', 't204', 't255', 't217', 't239']

to_drop = [f't{i}' for i in range(0,300) if f't{i}' not in sb_best_25[:10]]

In [44]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

columns = ['Summary', 'Episode_Title', 'Stars', 'Duration', 'Votes', 'text', 'match', 'len in words', 'len_prep'] + to_drop

X = books.drop(columns=columns)
X.Director = X.Director.apply(direr)
y = books.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['books_10_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.31665680752510633 0.270273378361764


### scriptbase

In [45]:
sb_best_25 = ['t134', 't103', 't12', 't194', 't276', 't35', 't180',
       't222', 't32', 't191', 't299', 't64', 't202', 't288', 't275', 't260',
       't84', 't182', 't165', 't210', 't90', 't204', 't255', 't217', 't239']

to_drop = [f't{i}' for i in range(0,300) if f't{i}' not in sb_best_25[:10]]

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

columns = ['Summary', 'Episode_Title', 'Stars', 'Duration', 'Votes', 'text', 'match', 'len in words', 'len_prep'] + to_drop

X = sb.drop(columns=columns)
X.Director = X.Director.apply(direr)
y = books.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['sb_10_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.29615088182111954 0.24775303050982417


In [46]:
resultlist

[['model', 'LOOCV mean score', 'std'],
 ['dummy_baseline', -0.3113667508113955, 0.24655333378118335],
 ['BOW_baseline', -0.31038842338807143, 0.2511842652329307],
 ['no_topic_baseline', -0.3200979000339483, 0.2584603535264025],
 ['no_topic_tf_idf_baseline', -0.31056740251808684, 0.2547505392051503],
 ['books_all_topics', -0.31670502401903533, 0.25496376558355],
 ['sc_all_topics', -0.3099117990860839, 0.24303544586694656],
 ['books_50_topics', -0.3025465625466021, 0.2652608993937459],
 ['sb_50_topics', -0.28075964835406053, 0.22891460909066416],
 ['books_25_topics', -0.3098606761309716, 0.24948065582815313],
 ['sb_25_topics', -0.2834794060697638, 0.2279225403433374],
 ['books_10_topics', -0.31665680752510633, 0.270273378361764],
 ['sb_10_topics', -0.29615088182111954, 0.24775303050982417]]